<a href="https://colab.research.google.com/github/Data-Scientist-Engineer/Face-Mask-detection/blob/main/VGG16_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.xception import Xception

from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
import datetime

from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

In [4]:
# Organize data into train, test directories
main_path = "/content/drive/MyDrive/python/Projet_CNN/model2/data"
with_mask_path = "/content/drive/MyDrive/python/Projet_CNN/model2/data/with_mask"
without_mask_path = "/content/drive/MyDrive/python/Projet_CNN/model2/data/without_mask"

In [5]:
from distutils.dir_util import copy_tree

if os.path.isdir('/content/drive/MyDrive/python/Projet_CNN/model2') is False:
        
    os.makedirs('/content/drive/MyDrive/python/Projet_CNN/model2')
    
    fromDir_main='/content/drive/MyDrive/python/Projet_CNN/model2'
    fromDir_with='/content/drive/MyDrive/python/Projet_CNN/model2/data/with_mask'
    fromDir_without='/content/drive/MyDrive/python/Projet_CNN/model2/data/without_mask'
    
    toDir_main= '/content/drive/MyDrive/python/Projet_CNN/data'
    toDir_with='/content/drive/MyDrive/python/Projet_CNN/model2/data/with_mask'
    toDir_without='/content/drive/MyDrive/python/Projet_CNN/model2/data/without_mask'
    
    
    fromDir_list=[fromDir_main,fromDir_with,fromDir_incorrect,fromDir_without]
    
    toDir_list=[toDir_main,toDir_with,toDir_incorrect,toDir_without]


    for fromDir,toDir in zip(fromDir_list,toDir_list):
        copy_tree(fromDir,toDir)

In [6]:
#Sanity Check
print(len(os.listdir("/content/drive/MyDrive/python/Projet_CNN/model2/data/with_mask")))
print(len(os.listdir("/content/drive/MyDrive/python/Projet_CNN/model2/data/without_mask")))

3725
3828


In [7]:
import os
train_size = 2000
test_size = 300
os.chdir(main_path)
if os.path.isdir('valid/with_mask') is False:
    #train:
    os.makedirs('/content/drive/MyDrive/python/Projet_CNN/model2/data/train/with_mask')
    os.makedirs('/content/drive/MyDrive/python/Projet_CNN/model2/data/train/without_mask')
    
    #test:
    os.makedirs('/content/drive/MyDrive/python/Projet_CNN/model2/data/test/with_mask')
    os.makedirs('/content/drive/MyDrive/python/Projet_CNN/model2/data/test/without_mask')

    
    os.chdir(with_mask_path)
    for i in random.sample(glob.glob('*'), train_size):
        shutil.move(i,
                    '/content/drive/MyDrive/python/Projet_CNN/model2/data/train/with_mask')

    os.chdir(without_mask_path)
    for i in random.sample(glob.glob('*'), train_size):
        shutil.move(i,
                    '/content/drive/MyDrive/python/Projet_CNN/model2/data/train/without_mask')
    
    os.chdir(with_mask_path)
    for i in random.sample(glob.glob('*'), test_size):
        shutil.move(i,
                    '/content/drive/MyDrive/python/Projet_CNN/model2/data/test/with_mask')

    os.chdir(without_mask_path)
    for i in random.sample(glob.glob('*'), test_size):
        shutil.move(i,
                    '/content/drive/MyDrive/python/Projet_CNN/model2/data/test/without_mask')
        
os.chdir('../../')

In [8]:
#Sanity Check
print(len(os.listdir('/content/drive/MyDrive/python/Projet_CNN/model2/data/train/with_mask')))
print(len(os.listdir('/content/drive/MyDrive/python/Projet_CNN/model2/data/train/without_mask')))

print(len(os.listdir('/content/drive/MyDrive/python/Projet_CNN/model2/data/test/with_mask')))
print(len(os.listdir('/content/drive/MyDrive/python/Projet_CNN/model2/data/test/without_mask')))

2000
2000
300
300


In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range=0.3,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   brightness_range=[0.4,1.5],
                                   horizontal_flip=True)

training_set = train_datagen.flow_from_directory(train_dir,
                                                batch_size=32,
                                                target_size=(224,224),
                                                shuffle=True,
                                                color_mode='rgb',
                                                class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(test_dir,
                                                batch_size=32,
                                                target_size=(224,224),
                                                shuffle=True,
                                                color_mode='rgb',
                                                class_mode='categorical')

Found 4000 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [11]:
vgg = VGG16(weights='imagenet', include_top=False,input_shape = (224,224,3))

58900480/58889256 [==============================] - 1s 0us/step


In [12]:
for layer in vgg.layers:
    layer.trainable = False

In [13]:
vgg16_model = Sequential()
vgg16_model.add(vgg)
vgg16_model.add(Flatten())
vgg16_model.add(Dense(2, activation = "softmax"))

vgg16_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 2)                 50178     
                                                                 
Total params: 14,764,866
Trainable params: 50,178
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
val_steps = test_set.n // test_set.batch_size
steps_per_epoch = training_set.n // training_set.batch_size

checkpoint = ModelCheckpoint("vgg16.h5",monitor = "val_accuracy",save_best_only = True,verbose=1)
earlystop = EarlyStopping(monitor="val_accuracy",patience=8,verbose=1)

sgd = SGD(lr=0.01, momentum=0.9, nesterov=True)
vgg16_model.compile(optimizer=sgd,loss = "categorical_crossentropy",metrics = ["accuracy"])

history = vgg16_model.fit(x=training_set,
                 validation_data=test_set,
                 epochs=25,
                 callbacks=[checkpoint,earlystop],
                 steps_per_epoch=steps_per_epoch,
                 validation_steps=val_steps)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/25
 69/125 [===============>..............] - ETA: 16:44 - loss: 11.3754 - accuracy: 0.6585

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


125/125 [==============================] - ETA: 0s - loss: 8.2842 - accuracy: 0.7178 
Epoch 00001: val_accuracy improved from -inf to 0.96007, saving model to vgg16.h5
125/125 [==============================] - 2560s 20s/step - loss: 8.2842 - accuracy: 0.7178 - val_loss: 0.4355 - val_accuracy: 0.9601
Epoch 2/25
125/125 [==============================] - ETA: 0s - loss: 2.4107 - accuracy: 0.8505 
Epoch 00002: val_accuracy did not improve from 0.96007
125/125 [==============================] - 2383s 19s/step - loss: 2.4107 - accuracy: 0.8505 - val_loss: 0.4541 - val_accuracy: 0.9566
Epoch 3/25
125/125 [==============================] - ETA: 0s - loss: 1.8386 - accuracy: 0.8633 
Epoch 00003: val_accuracy did not improve from 0.96007
125/125 [==============================] - 2382s 19s/step - loss: 1.8386 - accuracy: 0.8633 - val_loss: 1.2222 - val_accuracy: 0.9080
Epoch 4/25
 37/125 [=======>......................] - ETA: 24:02 - loss: 2.0609 - accuracy: 0.8598